In [14]:
import os

from PIL import Image
from points import NumberPoints
import random
from tqdm import tqdm

In [23]:
EASY = True
IMAGE_SET_NUM = 10
colour_1 = "yellow"
colour_2 = "blue"

In [16]:
random.seed(1714)

# constants
boundary_width = 5
background_colour = "#808080"
yellow = "#fffe04"
blue = "#0003f9"
point_sep = 20
min_point_radius = 8
max_point_radius = 16
init_size = 512
mode = "RGB"

IMG_DIR = "../images/output_pngs"
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(os.path.join(IMG_DIR, colour_1), exist_ok=True)
os.makedirs(os.path.join(IMG_DIR, colour_2), exist_ok=True)

In [17]:
# halberda point num and ratios
min_point_num = 4
max_point_num = 16
ratios = [2 / 5, 1 / 2, 2 / 3, 3 / 4]
if not EASY:
    ratios += [4 / 5, 5 / 6, 7 / 8, 8 / 9, 9 / 10, 10 / 11, 11 / 12]

In [18]:
p = [[(b, int(b / a)) for a in ratios if max_point_num >= b / a == round(b / a)] for b in range(min_point_num, max_point_num + 1)]
pos = [item for sublist in p for item in sublist]

In [24]:
def create_image(n1, n2, equalized):
    img = Image.new(mode, (init_size, init_size), color=background_colour)
    number_points = NumberPoints(img, init_size, yellow=yellow, blue=blue)
    point_array = number_points.design_n_points(n1, colour_1)
    point_array = number_points.design_n_points(n2, colour_2, point_array=point_array)
    if equalized:
        point_array = number_points.equalize_areas(point_array)

    return number_points.draw_points(point_array)
        
def create_and_save(n1, n2, equalized, tag=""):
    name = f"img_{n1}_{n2}_{tag}{'_equalized' if equalized else ''}v3.png"
    img = create_image(n1, n2, equalized)
    img.save(os.path.join(IMG_DIR, colour_1 if n1 > n2 else colour_2, name))


In [26]:
print(f"This will make {IMAGE_SET_NUM * len(pos) * 4} images.")

for i in tqdm(range(IMAGE_SET_NUM)):
    for pair in pos:
        create_and_save(pair[0], pair[1], equalized=True, tag=i)
        create_and_save(pair[1], pair[0], equalized=True, tag=i)
        create_and_save(pair[0], pair[1], equalized=False, tag=i)
        create_and_save(pair[1], pair[0], equalized=False, tag=i)

This will make 560 images.


  0%|          | 0/10 [07:03<?, ?it/s]


KeyboardInterrupt: 

In [21]:
pos

[(4, 10),
 (4, 8),
 (4, 6),
 (5, 10),
 (6, 15),
 (6, 12),
 (6, 9),
 (6, 8),
 (7, 14),
 (8, 16),
 (8, 12),
 (9, 12),
 (10, 15),
 (12, 16)]

In [22]:
len(pos)

14